#  CNN Hyperparameters

## COVIDx Dataset

In [13]:
from fastai.vision.all import *
from efficientnet_pytorch import EfficientNet 
path = Path('/home/jupyter/covidx')

In [14]:
torch.cuda.empty_cache()

In [15]:
# fix result 
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
SEED = 42
seed_everything(SEED)

In [16]:
df = pd.read_csv(path/'covidx.csv')
df

,patientid,path,finding,source,is_valid
0,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5.jpeg,pneumonia,cohen,False
1,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10.jpeg,pneumonia,cohen,False
2,7,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day7.jpeg,pneumonia,cohen,False
3,7,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day12.jpeg,pneumonia,cohen,False
4,9,SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day5.jpeg,pneumonia,cohen,False
...,...,...,...,...,...
15491,2c917d3a-95cb-4c11-802c-f83e28cb37bc,2c917d3a-95cb-4c11-802c-f83e28cb37bc.png,pneumonia,rsna,True
15492,3040d9d7-d895-453f-887c-616c10531960,3040d9d7-d895-453f-887c-616c10531960.png,pneumonia,rsna,True
15493,c07f52df-d481-434f-84c1-04263926ac40,c07f52df-d481-434f-84c1-04263926ac40.png,pneumonia,rsna,True
15494,c109061a-d815-4cae-8343-9230d8024adf,c109061a-d815-4cae-8343-9230d8024adf.png,pneumonia,rsna,True


In [17]:
get_x=lambda x:path/"images"/f"{x[1]}"
get_y=lambda x:x[2]
splitter=ColSplitter('is_valid')

In [18]:
metrics=[accuracy,  
         RocAuc(average='macro', multi_class='ovr'),
         MatthewsCorrCoef(sample_weight=None),
         Precision(average='macro'),
         Recall(average='macro'),     
         F1Score(average='macro')] 

In [19]:
item_tfms=Resize(480, method='squish', pad_mode='zeros', resamples=(2, 0))
batch_tfms=[*aug_transforms(mult=1.0, do_flip=False, flip_vert=False, 
                            max_rotate=20.0, max_zoom=1.2, max_lighting=0.3, max_warp=0.2, 
                            p_affine=0.75, p_lighting=0.75, 
                            xtra_tfms=None, size=None, mode='bilinear', pad_mode='reflection', 
                            align_corners=True, batch=False, min_scale=1.0),
                            Normalize.from_stats(*imagenet_stats)]

In [20]:
db = DataBlock(blocks=(ImageBlock(cls=PILImageBW), CategoryBlock),
               get_x=get_x,
               get_y=get_y,
               splitter=splitter, 
               item_tfms = item_tfms,
               batch_tfms=batch_tfms)

# VGG-16

## Epoch

## 10

In [29]:
from torchvision.models import vgg16
arch = vgg16
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [30]:
dl = db.dataloaders(df, bs=bs) 

In [31]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.510742,0.450031,0.842305,0.934196,0.705786,0.831642,0.727381,0.764304,02:16


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.339053,0.298428,0.894870,0.961443,0.802211,0.896132,0.779133,0.820200,05:15
1,0.295733,0.265340,0.909436,0.971165,0.829930,0.916149,0.792967,0.835692,05:13
2,0.242192,0.269334,0.905003,0.968360,0.821758,0.876673,0.777203,0.810588,05:13
3,0.231070,0.267211,0.917036,0.969705,0.844140,0.934098,0.799697,0.843137,05:13
4,0.210688,0.222854,0.920203,0.980668,0.850174,0.928878,0.824311,0.863131,05:13
5,0.185515,0.243568,0.919569,0.978860,0.849120,0.941325,0.790668,0.835315,05:13
6,0.159219,0.212531,0.921469,0.983590,0.853345,0.937411,0.836333,0.873719,05:13
7,0.137119,0.210282,0.923369,0.985714,0.856301,0.944394,0.824898,0.866906,05:13
8,0.141596,0.210742,0.922103,0.985479,0.854434,0.942563,0.830796,0.870732,05:13
9,0.137846,0.210723,0.920836,0.985413,0.852005,0.941777,0.829858,0.869881,05:13


## 20

In [21]:
from torchvision.models import vgg16
arch = vgg16
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [22]:
dl = db.dataloaders(df, bs=bs) 

In [23]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [24]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.511293,0.455460,0.841039,0.933430,0.703682,0.823954,0.726259,0.761197,01:42


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.343960,0.299327,0.891704,0.961256,0.796139,0.883622,0.773924,0.812834,05:45
1,0.304951,0.265173,0.905636,0.967821,0.822231,0.914622,0.781648,0.824163,05:42
2,0.250443,0.276168,0.905636,0.963317,0.822589,0.882295,0.777027,0.812171,05:42
3,0.249550,0.267995,0.914503,0.971617,0.839178,0.926660,0.791539,0.834654,05:42
4,0.236230,0.252162,0.912603,0.974768,0.835902,0.920341,0.800017,0.840461,05:42
5,0.216017,0.257613,0.917036,0.975449,0.844180,0.933296,0.785651,0.829453,05:42
6,0.195917,0.236494,0.919569,0.978131,0.849990,0.922691,0.835388,0.868624,05:42
7,0.166595,0.204610,0.927169,0.982905,0.863769,0.926851,0.856171,0.885427,05:42
8,0.169070,0.208057,0.927802,0.984147,0.866420,0.934492,0.877056,0.901115,05:42
9,0.152626,0.246903,0.926536,0.983272,0.862084,0.942666,0.820130,0.862956,05:42


## 40

In [33]:
from torchvision.models import vgg16
arch = vgg16
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [34]:
dl = db.dataloaders(df, bs=bs) 

In [35]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [36]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.510780,0.452044,0.838505,0.934659,0.698799,0.819837,0.718285,0.753869,01:42


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.343122,0.303824,0.891704,0.960134,0.796135,0.883102,0.770968,0.810245,05:42
1,0.311995,0.275098,0.904370,0.964615,0.819808,0.903895,0.779603,0.820707,05:42
2,0.254997,0.268435,0.903103,0.965450,0.817903,0.877905,0.791599,0.823743,05:42
3,0.253303,0.265461,0.908170,0.972686,0.826931,0.921514,0.774469,0.818607,05:42
4,0.238184,0.255328,0.913870,0.974882,0.838342,0.921363,0.803727,0.843811,05:42
5,0.218890,0.281181,0.908803,0.973006,0.828108,0.922358,0.771336,0.816078,05:42
6,0.209834,0.256274,0.916403,0.978207,0.843183,0.922392,0.800058,0.840471,05:42
7,0.184822,0.219045,0.920836,0.979116,0.853074,0.890209,0.861459,0.874526,05:42
8,0.194802,0.218297,0.926536,0.981123,0.863443,0.924672,0.855052,0.883108,05:43
9,0.180404,0.280332,0.919569,0.980657,0.850474,0.943299,0.819321,0.864556,05:42


## Batch Size

## 8

In [9]:
from torchvision.models import vgg16
arch = vgg16
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.532515,0.460921,0.840405,0.924718,0.698848,0.809194,0.702409,0.737987,02:30


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.371442,0.292244,0.898670,0.967134,0.810024,0.878894,0.802090,0.832976,06:32
1,0.338025,0.272986,0.901203,0.970167,0.814036,0.904854,0.797125,0.836274,06:31
2,0.333592,0.269327,0.906270,0.971984,0.823517,0.913325,0.774077,0.816217,06:31
3,0.284895,0.263382,0.917669,0.972186,0.846939,0.927983,0.811527,0.850318,06:31
4,0.295700,0.247369,0.913870,0.976243,0.838304,0.921529,0.824244,0.861707,06:31
5,0.280971,0.236876,0.917036,0.978106,0.844150,0.925872,0.818733,0.858033,06:31
6,0.294591,0.273493,0.918936,0.979964,0.848415,0.940272,0.816902,0.858717,06:31
7,0.224852,0.249325,0.922736,0.976462,0.855052,0.946527,0.830624,0.873419,06:31
8,0.293905,0.276461,0.918936,0.980611,0.847513,0.942607,0.793991,0.840315,06:31
9,0.306980,0.211770,0.936669,0.984149,0.881710,0.944512,0.872910,0.903228,06:31


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

## 16

In [9]:
from torchvision.models import vgg16
arch = vgg16
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.492169,0.418506,0.854972,0.937577,0.723893,0.898769,0.699232,0.744341,01:47


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.331609,0.288821,0.892337,0.964827,0.797418,0.882226,0.790007,0.825052,05:20
1,0.300835,0.272047,0.901837,0.968518,0.815244,0.895612,0.791958,0.829571,05:19
2,0.282888,0.246519,0.906903,0.975995,0.824785,0.916375,0.791641,0.833283,05:19
3,0.239707,0.255124,0.910703,0.976675,0.831994,0.919314,0.796857,0.838368,05:19
4,0.242813,0.248942,0.916403,0.976987,0.843230,0.919248,0.822424,0.859706,05:19
5,0.236638,0.265165,0.905636,0.979332,0.825479,0.930456,0.781086,0.822885,05:19
6,0.266934,0.199539,0.929702,0.982440,0.868817,0.923848,0.877821,0.898392,05:19
7,0.185905,0.204543,0.924003,0.982654,0.859156,0.887321,0.886631,0.886669,05:19
8,0.220586,0.201779,0.927169,0.986042,0.864698,0.925738,0.861342,0.887783,05:19
9,0.221229,0.230646,0.925269,0.981892,0.860945,0.942606,0.853385,0.889712,05:19


In [13]:
learn.summary()

Sequential (Input shape: ['16 x 3 x 480 x 480'])
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               16 x 64 x 480 x 480  1,792      True      
________________________________________________________________
ReLU                 16 x 64 x 480 x 480  0          False     
________________________________________________________________
Conv2d               16 x 64 x 480 x 480  36,928     True      
________________________________________________________________
ReLU                 16 x 64 x 480 x 480  0          False     
________________________________________________________________
MaxPool2d            16 x 64 x 240 x 240  0          False     
________________________________________________________________
Conv2d               16 x 128 x 240 x 24  73,856     True      
________________________________________________________________
ReLU                 16 x 128 x 240 x 24  0          False     
_________________________________________________

## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [9]:
from torchvision.models import vgg16
arch = vgg16
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.06, reduction='mean', flatten=True, floatify=False, is_2d=True)
bs = 32
epoch = 30 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.624632,0.571274,0.834706,0.932476,0.691044,0.827837,0.703461,0.743840,02:22


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.475750,0.432638,0.886637,0.960787,0.786097,0.885514,0.752802,0.795018,05:19
1,0.445722,0.411327,0.903103,0.962870,0.817205,0.904936,0.757969,0.800701,05:16
2,0.403005,0.407296,0.901837,0.963032,0.815157,0.882608,0.767931,0.804694,05:16
3,0.399587,0.391150,0.914503,0.967370,0.839332,0.926535,0.795049,0.837486,05:16
4,0.390226,0.391264,0.914503,0.969459,0.839323,0.912730,0.803181,0.841709,05:16
5,0.372083,0.398417,0.909436,0.970375,0.829266,0.926859,0.749720,0.792832,05:16
6,0.368195,0.368349,0.922103,0.977901,0.853706,0.928558,0.814168,0.854512,05:16
7,0.343563,0.353533,0.925269,0.981351,0.859973,0.923266,0.834529,0.868766,05:16
8,0.350066,0.371346,0.917036,0.983503,0.847752,0.917083,0.847922,0.874598,05:16
9,0.342469,0.381374,0.919569,0.972152,0.849569,0.940739,0.806014,0.852295,05:16


# VGG-19

## Epoch

### 10

In [10]:
from torchvision.models import vgg19
arch = vgg19
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [11]:
dl = db.dataloaders(df, bs=bs) 

In [12]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [13]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.512763,0.575574,0.800507,0.916905,0.627626,0.869889,0.630283,0.678245,02:26


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.345652,0.350802,0.875871,0.951683,0.765105,0.916165,0.708705,0.754942,05:59
1,0.298669,0.313865,0.900570,0.962252,0.813089,0.932824,0.765152,0.814745,05:57
2,0.251678,0.345787,0.890437,0.967204,0.796302,0.920549,0.703120,0.732858,05:57
3,0.237077,0.241135,0.918936,0.976156,0.849501,0.934449,0.824107,0.862387,05:57
4,0.204525,0.295797,0.911336,0.977658,0.832861,0.937447,0.753806,0.798924,05:56
5,0.181282,0.215415,0.929069,0.981023,0.867266,0.943082,0.832536,0.872186,05:56
6,0.151480,0.229072,0.919569,0.982772,0.850611,0.920531,0.821896,0.856868,05:56
7,0.125158,0.209764,0.930969,0.982909,0.870593,0.935995,0.832744,0.870939,05:56
8,0.133960,0.221096,0.925269,0.983585,0.860091,0.940110,0.823994,0.864575,05:57
9,0.131595,0.213597,0.925269,0.983764,0.860404,0.935550,0.836190,0.872943,05:57


## 20

In [22]:
from torchvision.models import vgg19
arch = vgg19
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [23]:
dl = db.dataloaders(df, bs=bs) 

In [24]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [25]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.513827,0.569283,0.801140,0.918787,0.629499,0.871889,0.639346,0.689658,01:56


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.345783,0.349606,0.876504,0.952493,0.766174,0.907088,0.715548,0.761486,06:36
1,0.306935,0.324300,0.894870,0.960719,0.801074,0.925785,0.725909,0.769221,06:36
2,0.263780,0.310835,0.899303,0.969582,0.810041,0.927180,0.725584,0.765470,06:35
3,0.249006,0.273646,0.912603,0.973277,0.835443,0.931891,0.785238,0.831062,06:35
4,0.231158,0.287356,0.915136,0.973855,0.840218,0.939920,0.780089,0.826506,06:35
5,0.203800,0.232868,0.917036,0.982450,0.846440,0.938472,0.812257,0.852843,06:35
6,0.185388,0.247593,0.924003,0.982435,0.858103,0.924453,0.826566,0.862200,06:35
7,0.156467,0.226521,0.934136,0.982647,0.877192,0.935075,0.858100,0.890068,06:35
8,0.166041,0.239208,0.930969,0.982528,0.870597,0.941402,0.838657,0.877171,06:35
9,0.167254,0.215911,0.934136,0.983931,0.876978,0.950735,0.849784,0.889134,06:35


## 40

In [34]:
from torchvision.models import vgg19
arch = vgg19
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [35]:
dl = db.dataloaders(df, bs=bs) 

In [36]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [37]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.513827,0.569283,0.801140,0.918787,0.629499,0.871889,0.639346,0.689658,01:56


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.347262,0.351249,0.875871,0.952179,0.765090,0.907313,0.720716,0.767723,06:38
1,0.309677,0.327505,0.894237,0.958682,0.799876,0.924908,0.726086,0.768925,06:38
2,0.266825,0.291021,0.901203,0.969493,0.813419,0.930114,0.754436,0.800849,06:38
3,0.252499,0.283557,0.903737,0.973044,0.818452,0.925813,0.760749,0.807366,06:38
4,0.237896,0.285430,0.911970,0.975342,0.834402,0.936517,0.764529,0.809124,06:38
5,0.211995,0.249421,0.916403,0.977143,0.842667,0.934219,0.786386,0.831600,06:38
6,0.197449,0.304394,0.912603,0.977029,0.837080,0.928432,0.773033,0.814653,06:38
7,0.182045,0.219596,0.925902,0.979663,0.861061,0.929321,0.837124,0.872883,06:38
8,0.195907,0.254012,0.918936,0.980355,0.847736,0.943225,0.822081,0.865727,06:39
9,0.191528,0.238617,0.925902,0.980519,0.861039,0.938253,0.818277,0.860926,06:38


## Batch Size

## 8

In [9]:
from torchvision.models import vgg19
arch = vgg19
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.503848,0.539405,0.811906,0.919023,0.643784,0.833619,0.651653,0.695613,03:00


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.370595,0.342144,0.885370,0.964052,0.783268,0.913506,0.737265,0.784476,07:42
1,0.350829,0.277218,0.912603,0.970394,0.836064,0.919245,0.797060,0.837386,07:41
2,0.319036,0.296304,0.901203,0.972458,0.814935,0.919486,0.746668,0.787327,07:42
3,0.298122,0.282241,0.913870,0.975263,0.837853,0.937939,0.776194,0.822097,07:40
4,0.245551,0.236334,0.923369,0.978886,0.856531,0.908693,0.841531,0.869353,07:39
5,0.270513,0.274220,0.917036,0.980541,0.843872,0.941774,0.774383,0.821473,07:40
6,0.306692,0.276432,0.910703,0.980151,0.835368,0.933946,0.802577,0.843155,07:42
7,0.211850,0.265387,0.920203,0.979828,0.850300,0.939977,0.813596,0.858069,07:41
8,0.305428,0.209828,0.929702,0.986223,0.868261,0.941691,0.842156,0.880473,07:41
9,0.286709,0.231535,0.925902,0.983602,0.864340,0.896418,0.883887,0.888355,07:41


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 16

In [21]:
from torchvision.models import vgg19
arch = vgg19
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [22]:
dl = db.dataloaders(df, bs=bs) 

In [23]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [24]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.490926,0.481258,0.835972,0.937052,0.691422,0.873299,0.700146,0.749819,02:02


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.338227,0.312358,0.892970,0.964602,0.798867,0.916052,0.776343,0.823027,06:07
1,0.305570,0.295478,0.906903,0.969326,0.824774,0.932442,0.774639,0.820049,06:05
2,0.265399,0.249290,0.916403,0.974745,0.842829,0.929838,0.809117,0.851334,06:05
3,0.244584,0.242459,0.918936,0.978704,0.848713,0.935180,0.826141,0.864911,06:05
4,0.225054,0.259202,0.913236,0.980502,0.836901,0.928656,0.793931,0.838806,06:05
5,0.243469,0.256998,0.913236,0.975671,0.838101,0.936478,0.773779,0.816690,06:05
6,0.244441,0.206484,0.936669,0.981191,0.881589,0.943417,0.874201,0.903482,06:05
7,0.169494,0.175001,0.935402,0.987013,0.879341,0.945690,0.882687,0.909737,06:06
8,0.234485,0.207037,0.929069,0.983515,0.867004,0.940145,0.839377,0.877648,06:06
9,0.195484,0.214085,0.935402,0.983243,0.879345,0.943300,0.862733,0.895960,06:06


## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [18]:
from torchvision.models import vgg19
arch = vgg19
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.06, reduction='mean', flatten=True, floatify=False, is_2d=True)
bs = 32
epoch = 30 

In [19]:
dl = db.dataloaders(df, bs=bs) 

In [20]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [21]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.625392,0.643686,0.801140,0.915580,0.630782,0.854939,0.635835,0.683660,02:30


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.480348,0.470174,0.874604,0.948853,0.762155,0.904129,0.705917,0.749620,07:35
1,0.444943,0.446375,0.893604,0.953487,0.798613,0.924619,0.719796,0.761296,07:32
2,0.411126,0.428457,0.892337,0.962608,0.796317,0.922899,0.705366,0.741346,07:32
3,0.399438,0.393506,0.911970,0.971147,0.834176,0.937468,0.782458,0.829274,07:32
4,0.380970,0.423361,0.903103,0.971248,0.816979,0.932271,0.726552,0.768441,07:32
5,0.365460,0.373199,0.921469,0.974380,0.853041,0.936344,0.801406,0.843833,07:32
6,0.353236,0.395918,0.907536,0.978083,0.826076,0.925213,0.755795,0.798429,07:32
7,0.335846,0.347663,0.934769,0.984500,0.878719,0.927696,0.873076,0.897128,07:32
8,0.345510,0.359970,0.927169,0.979280,0.863462,0.943205,0.841757,0.880211,07:32
9,0.340648,0.394996,0.917036,0.971104,0.844676,0.938359,0.786767,0.834547,07:32


# ResNet-18

## Epoch

### 10

In [35]:
arch = resnet18
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [36]:
dl = db.dataloaders(df, bs=bs) 

In [37]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [38]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.481022,0.360238,0.873338,0.950756,0.760677,0.894132,0.736211,0.782834,01:03


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.289795,0.255401,0.913870,0.969528,0.838408,0.931171,0.795226,0.838215,01:18
1,0.259806,0.204938,0.923369,0.983037,0.856391,0.932706,0.849110,0.882921,01:18
2,0.228260,0.230879,0.921469,0.982841,0.852419,0.938856,0.806955,0.851314,01:18
3,0.205904,0.245011,0.918303,0.982958,0.848859,0.939254,0.807282,0.848412,01:18
4,0.195957,0.219952,0.922103,0.983533,0.853651,0.939988,0.813799,0.857414,01:18
5,0.150547,0.191056,0.925269,0.986863,0.860547,0.935792,0.844875,0.879626,01:18
6,0.123753,0.186733,0.933502,0.990391,0.875570,0.948079,0.866589,0.899783,01:17
7,0.095947,0.188320,0.934769,0.991100,0.877915,0.949648,0.866974,0.900723,01:18
8,0.088533,0.215235,0.932236,0.989445,0.873222,0.950477,0.849018,0.887277,01:18
9,0.074958,0.216634,0.930969,0.989215,0.870805,0.949759,0.845308,0.884262,01:18


## 20

In [47]:
arch = resnet18
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [51]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [52]:
dl = db.dataloaders(df, bs=bs) 

In [53]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.489349,0.371563,0.863838,0.956468,0.746278,0.868462,0.777139,0.812445,01:04


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.290796,0.253645,0.920836,0.971056,0.851652,0.927861,0.828567,0.865568,01:18
1,0.250388,0.253592,0.920836,0.973134,0.851226,0.944569,0.809535,0.855045,01:18
2,0.239048,0.255096,0.914503,0.978090,0.839183,0.938514,0.786179,0.831716,01:18
3,0.226221,0.257435,0.915136,0.981032,0.840266,0.939158,0.785633,0.831758,01:19
4,0.227785,0.202231,0.927802,0.986927,0.864664,0.948002,0.836774,0.877435,01:19
5,0.177434,0.259834,0.910070,0.986474,0.833770,0.933744,0.781688,0.823385,01:19
6,0.179825,0.162109,0.939835,0.989267,0.887645,0.952028,0.880333,0.910264,01:19
7,0.153564,0.168612,0.941102,0.989414,0.890000,0.954880,0.891806,0.919192,01:19
8,0.152684,0.170960,0.941102,0.990595,0.890278,0.944672,0.879421,0.906728,01:19
9,0.131773,0.181240,0.944269,0.988262,0.896117,0.957908,0.898680,0.924788,01:19


## 40

In [9]:
arch = resnet18
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.481022,0.360238,0.873338,0.950756,0.760677,0.894132,0.736211,0.782834,02:23


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.293793,0.261279,0.908170,0.971709,0.828500,0.932464,0.783889,0.827240,01:16
1,0.252657,0.233091,0.916403,0.980882,0.842863,0.933748,0.796179,0.840033,01:16
2,0.211453,0.239549,0.913870,0.981482,0.837829,0.931733,0.778966,0.823984,01:16
3,0.214934,0.232315,0.920203,0.981903,0.849970,0.942879,0.803430,0.848915,01:16
4,0.208239,0.225608,0.919569,0.983686,0.849451,0.940857,0.799907,0.843603,01:16
5,0.183196,0.251956,0.918303,0.982018,0.846483,0.934578,0.786405,0.830487,01:16
6,0.167028,0.206587,0.925269,0.986984,0.859872,0.932820,0.839335,0.875590,01:16
7,0.161546,0.202991,0.937935,0.985740,0.884120,0.953417,0.873478,0.906765,01:16
8,0.166080,0.201965,0.930336,0.986189,0.870263,0.931848,0.860269,0.889279,01:16
9,0.163605,0.192531,0.935402,0.988130,0.879028,0.948758,0.847760,0.885992,01:16


## Batch Size

## 8

In [21]:
arch = resnet18
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [22]:
dl = db.dataloaders(df, bs=bs) 

In [23]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [24]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.501623,0.458977,0.830272,0.948267,0.681478,0.861817,0.677355,0.725250,01:24


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.287950,0.252539,0.918303,0.975228,0.846712,0.919008,0.828725,0.863704,01:46
1,0.325409,0.251401,0.915769,0.982273,0.842046,0.937305,0.815396,0.859406,01:45
2,0.310275,0.219640,0.920203,0.985074,0.850428,0.936191,0.819505,0.860391,01:44
3,0.286677,0.237407,0.920203,0.978873,0.851543,0.931214,0.833361,0.869189,01:44
4,0.287409,0.274241,0.917669,0.978047,0.845345,0.937375,0.821328,0.863614,01:45
5,0.264631,0.264801,0.922103,0.981000,0.853803,0.943719,0.798273,0.843032,01:44
6,0.316085,0.250654,0.910070,0.979064,0.833579,0.911710,0.835836,0.865073,01:44
7,0.193333,0.242060,0.928436,0.978540,0.865884,0.941367,0.841403,0.879909,01:44
8,0.303499,0.203303,0.931602,0.983837,0.872232,0.933751,0.865459,0.893925,01:44
9,0.288293,0.186601,0.937302,0.984900,0.883095,0.949184,0.872179,0.904483,01:44


## 16

In [33]:
arch = resnet18
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [34]:
dl = db.dataloaders(df, bs=bs) 

In [35]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [36]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.445460,0.453761,0.849905,0.949096,0.716552,0.893457,0.686619,0.734405,01:08


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.280518,0.285992,0.911970,0.967529,0.834518,0.935869,0.770073,0.814852,01:26
1,0.254928,0.258296,0.911970,0.980093,0.834640,0.936186,0.799271,0.843501,01:27
2,0.247593,0.228948,0.923369,0.983615,0.856320,0.944282,0.830442,0.871511,01:27
3,0.210294,0.258443,0.918936,0.982466,0.848172,0.935103,0.818936,0.859537,01:27
4,0.228938,0.233236,0.922103,0.985526,0.853754,0.944046,0.820635,0.863615,01:27
5,0.215222,0.297448,0.911336,0.977578,0.834220,0.934847,0.756943,0.798916,01:27
6,0.214325,0.196638,0.931602,0.987900,0.872610,0.940665,0.855847,0.888866,01:27
7,0.186707,0.198110,0.929702,0.983981,0.869095,0.923034,0.866183,0.891053,01:27
8,0.200782,0.169552,0.942369,0.989351,0.892746,0.940521,0.902721,0.919951,01:27
9,0.194868,0.155147,0.945535,0.990292,0.898533,0.954756,0.914955,0.933289,01:27


## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [45]:
arch = resnet18
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.06, reduction='mean', flatten=True, floatify=False, is_2d=True)
bs = 32
epoch = 30 

In [46]:
dl = db.dataloaders(df, bs=bs) 

In [49]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [50]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.599050,0.500697,0.860038,0.945438,0.735444,0.899495,0.710197,0.759307,01:03


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.434049,0.407275,0.909436,0.966943,0.830444,0.933760,0.775772,0.819588,01:17
1,0.398571,0.375847,0.921469,0.978357,0.852534,0.933720,0.822108,0.862557,01:18
2,0.368496,0.381206,0.913236,0.980327,0.836593,0.931213,0.775448,0.820560,01:18
3,0.362043,0.382214,0.918936,0.975827,0.847975,0.940684,0.798793,0.843026,01:17
4,0.356695,0.374193,0.918303,0.979365,0.846339,0.941251,0.788254,0.834121,01:18
5,0.343214,0.364008,0.917036,0.979109,0.844724,0.939295,0.789900,0.833812,01:18
6,0.333856,0.350571,0.924636,0.987033,0.858899,0.939777,0.826390,0.866550,01:18
7,0.321817,0.325510,0.944269,0.988309,0.895996,0.951628,0.886664,0.914257,01:18
8,0.325986,0.330341,0.941735,0.988891,0.892235,0.938988,0.904005,0.919685,01:18
9,0.317785,0.340534,0.936669,0.982863,0.881387,0.949774,0.848145,0.886682,01:18


# ResNet-34

## Epoch

### 10

In [59]:
arch = resnet34
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [60]:
dl = db.dataloaders(df, bs=bs) 

In [61]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [62]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.499208,0.390638,0.863205,0.941087,0.741082,0.893478,0.719654,0.767465,01:20


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.290187,0.305898,0.901837,0.962731,0.814614,0.931365,0.745573,0.791662,01:50
1,0.264677,0.284842,0.905003,0.971150,0.822263,0.937693,0.764832,0.815174,01:50
2,0.235827,0.260692,0.902470,0.979944,0.816902,0.918853,0.729497,0.767102,01:50
3,0.215778,0.209650,0.932236,0.982777,0.873739,0.944717,0.833308,0.872341,01:50
4,0.194892,0.208388,0.930969,0.984361,0.871934,0.948350,0.830152,0.870156,01:50
5,0.138688,0.222430,0.919569,0.986170,0.849833,0.941044,0.792513,0.835902,01:50
6,0.113924,0.151177,0.947435,0.991603,0.902172,0.957711,0.912385,0.932731,01:50
7,0.077797,0.159070,0.948068,0.992297,0.903004,0.962477,0.885967,0.917697,01:50
8,0.071268,0.174258,0.946168,0.991422,0.899481,0.961732,0.895557,0.924042,01:50
9,0.051183,0.179180,0.940469,0.991277,0.888872,0.956344,0.883851,0.914097,01:50


## 20

In [9]:
arch = resnet34
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [12]:
dl = db.dataloaders(df, bs=bs) 

In [13]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [14]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.499208,0.390638,0.863205,0.941087,0.741082,0.893478,0.719654,0.767465,02:08


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.289284,0.279059,0.903103,0.970174,0.817190,0.924725,0.770165,0.816020,01:48
1,0.253357,0.226204,0.917036,0.977564,0.844317,0.934071,0.805979,0.848491,01:49
2,0.217993,0.272541,0.912603,0.973866,0.836511,0.928147,0.763610,0.805254,01:48
3,0.223090,0.282854,0.924636,0.972814,0.858449,0.946020,0.784996,0.830536,01:48
4,0.218146,0.209630,0.927802,0.983790,0.864753,0.935821,0.834006,0.872805,01:48
5,0.179482,0.229059,0.929702,0.982644,0.868066,0.950414,0.814066,0.859171,01:48
6,0.168487,0.198464,0.933502,0.988053,0.876642,0.942904,0.883771,0.908729,01:48
7,0.135320,0.231932,0.920836,0.985450,0.856706,0.877877,0.908410,0.887883,01:48
8,0.151614,0.237093,0.923369,0.988503,0.856575,0.944180,0.826005,0.867302,01:48
9,0.122721,0.204348,0.935402,0.989092,0.879125,0.955525,0.851829,0.891819,01:48


## 40

In [23]:
arch = resnet34
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [24]:
dl = db.dataloaders(df, bs=bs) 

In [25]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [26]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.499208,0.390638,0.863205,0.941087,0.741082,0.893478,0.719654,0.767465,01:17


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.290935,0.267013,0.908803,0.971880,0.828188,0.935392,0.789260,0.835615,01:48
1,0.250672,0.237744,0.913870,0.980045,0.838004,0.937764,0.788390,0.833810,01:48
2,0.208853,0.261905,0.905003,0.980397,0.821124,0.931305,0.730450,0.769935,01:48
3,0.203796,0.247385,0.925902,0.981924,0.861651,0.951234,0.819942,0.866264,01:48
4,0.192794,0.232910,0.922736,0.980426,0.855081,0.943959,0.810292,0.854063,01:48
5,0.164455,0.308551,0.906903,0.966840,0.826468,0.931802,0.754860,0.796149,01:48
6,0.166842,0.208877,0.927169,0.988223,0.865626,0.937133,0.865775,0.894117,01:48
7,0.155070,0.230695,0.928436,0.984052,0.869300,0.905379,0.885024,0.893184,01:48
8,0.181424,0.162117,0.947435,0.986364,0.902204,0.938717,0.909060,0.922895,01:48
9,0.170170,0.238779,0.924003,0.978647,0.859575,0.886100,0.876101,0.880203,01:48


## Batch Size

## 8

In [35]:
arch = resnet34
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [36]:
dl = db.dataloaders(df, bs=bs) 

In [37]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [38]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.514931,0.448203,0.825206,0.938253,0.670677,0.879896,0.679517,0.730926,01:51


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.311786,0.265436,0.911336,0.974837,0.833378,0.918443,0.804628,0.845594,02:26
1,0.315250,0.284818,0.906903,0.981409,0.826041,0.932801,0.780561,0.829367,02:27
2,0.285012,0.320984,0.888537,0.977858,0.803162,0.914942,0.792162,0.826961,02:27
3,0.278535,0.212793,0.932236,0.984592,0.873007,0.941323,0.836454,0.875240,02:26
4,0.300672,0.231333,0.928436,0.983397,0.866591,0.952988,0.856744,0.895365,02:26
5,0.265481,0.280097,0.917669,0.978940,0.845342,0.943399,0.788436,0.836365,02:25
6,0.297704,0.205459,0.932869,0.985963,0.875915,0.917610,0.899659,0.908041,02:26
7,0.191659,0.266394,0.922103,0.979254,0.853927,0.933353,0.823776,0.863230,02:27
8,0.266286,0.256499,0.913870,0.980883,0.838156,0.937520,0.800217,0.844652,02:26
9,0.282082,0.192950,0.937302,0.982791,0.883336,0.933130,0.889919,0.909539,02:26


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## 16

In [47]:
arch = resnet34
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [48]:
dl = db.dataloaders(df, bs=bs) 

In [49]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [50]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.459629,0.438516,0.856238,0.925264,0.726471,0.890935,0.671345,0.710120,01:28


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.276732,0.273092,0.914503,0.969289,0.839346,0.935302,0.794684,0.840918,02:00
1,0.255508,0.210909,0.925269,0.983422,0.860029,0.944112,0.837859,0.878426,02:02
2,0.225354,0.238573,0.915769,0.983619,0.844612,0.937270,0.846246,0.880200,02:00
3,0.209985,0.204676,0.934769,0.982658,0.878412,0.939320,0.868634,0.897669,02:02
4,0.228344,0.242609,0.924636,0.981385,0.859970,0.947480,0.847095,0.886869,02:00
5,0.206224,0.208796,0.930969,0.984799,0.872163,0.948215,0.847707,0.884313,02:00
6,0.231683,0.166451,0.941735,0.990101,0.892016,0.932224,0.919715,0.925766,02:00
7,0.176214,0.294698,0.898670,0.977733,0.822142,0.807135,0.891726,0.827863,01:59
8,0.204067,0.161486,0.941102,0.991252,0.890542,0.951931,0.891063,0.916902,01:58
9,0.210021,0.154873,0.941102,0.989198,0.890274,0.931768,0.894024,0.911374,01:58


## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [59]:
arch = resnet34
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.06, reduction='mean', flatten=True, floatify=False, is_2d=True)
bs = 32
epoch = 30 

In [60]:
dl = db.dataloaders(df, bs=bs) 

In [61]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [62]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.610142,0.509555,0.855605,0.940238,0.726552,0.896284,0.701647,0.749983,01:19


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.431900,0.408685,0.906270,0.971647,0.823201,0.932973,0.755968,0.801374,01:49
1,0.394922,0.380159,0.918936,0.979210,0.847527,0.936616,0.781795,0.827746,01:49
2,0.366294,0.386234,0.906903,0.977606,0.824330,0.933691,0.738789,0.781767,01:49
3,0.355152,0.367806,0.929702,0.971955,0.868111,0.950802,0.822198,0.866675,01:49
4,0.355369,0.345840,0.934136,0.985615,0.876658,0.954705,0.842574,0.884407,01:49
5,0.329839,0.372761,0.923369,0.974977,0.856306,0.944328,0.793666,0.838321,01:49
6,0.332092,0.362123,0.921469,0.987380,0.855368,0.931864,0.836329,0.870546,01:49
7,0.313589,0.351491,0.934769,0.985666,0.880554,0.900140,0.908745,0.902725,01:49
8,0.328114,0.326563,0.939202,0.990553,0.887318,0.943041,0.887715,0.911231,01:49
9,0.322049,0.333818,0.937302,0.983353,0.883237,0.954039,0.863309,0.900054,01:49


# ResNet-50

## Epoch

### 10

In [9]:
arch = resnet50
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.443581,0.356874,0.872071,0.951454,0.760275,0.838608,0.768724,0.796778,02:31


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.277865,0.270858,0.908170,0.962480,0.827007,0.927311,0.775023,0.819811,03:22
1,0.250054,0.232127,0.928436,0.972853,0.867148,0.920053,0.865421,0.888433,03:22
2,0.230650,0.258470,0.910703,0.981278,0.840662,0.921589,0.857827,0.881143,03:21
3,0.216328,0.182553,0.930969,0.985341,0.871729,0.934065,0.883930,0.905735,03:21
4,0.201755,0.213515,0.923369,0.984954,0.859844,0.933678,0.852796,0.883030,03:21
5,0.149409,0.187308,0.938569,0.989363,0.884958,0.956746,0.846503,0.887492,03:21
6,0.103060,0.140181,0.951235,0.992014,0.909376,0.963219,0.933128,0.947300,03:21
7,0.084955,0.140832,0.948702,0.993145,0.904337,0.963472,0.911847,0.934967,03:21
8,0.066165,0.149510,0.948068,0.993536,0.903122,0.962037,0.900566,0.927165,03:21
9,0.064791,0.153118,0.949335,0.993232,0.905544,0.962544,0.898916,0.926160,03:21


## 20

In [11]:
from torchvision.models import vgg16
arch = vgg16
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [12]:
dl = db.dataloaders(df, bs=bs) 

In [13]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [14]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.443581,0.356874,0.872071,0.951454,0.760275,0.838608,0.768724,0.796778,02:42


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.273871,0.253506,0.915769,0.966525,0.841984,0.933068,0.805226,0.847601,03:20
1,0.234378,0.219782,0.922103,0.976593,0.853875,0.931089,0.836714,0.873375,03:19
2,0.221047,0.220819,0.923369,0.983338,0.856061,0.928070,0.803648,0.845132,03:19
3,0.220235,0.216384,0.931602,0.977322,0.871954,0.941468,0.850860,0.886235,03:19
4,0.220627,0.250888,0.915136,0.977769,0.844110,0.920266,0.805952,0.842402,03:19
5,0.191360,0.281948,0.918936,0.978425,0.847691,0.941315,0.760725,0.804316,03:19
6,0.164263,0.197406,0.939835,0.986959,0.887566,0.952469,0.877376,0.908489,03:19
7,0.138198,0.184772,0.930969,0.987066,0.874086,0.862627,0.912025,0.883208,03:19
8,0.145956,0.169538,0.946168,0.989987,0.899707,0.956877,0.900174,0.924830,03:20
9,0.129338,0.189172,0.933502,0.989894,0.875504,0.949555,0.847742,0.886982,03:19


## 40

In [23]:
arch = resnet50
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [24]:
dl = db.dataloaders(df, bs=bs) 

In [25]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [26]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.443581,0.356874,0.872071,0.951454,0.760275,0.838608,0.768724,0.796778,02:26


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.266151,0.268639,0.912603,0.964021,0.836220,0.929644,0.786156,0.829107,03:19
1,0.234758,0.221854,0.927169,0.977397,0.863366,0.938735,0.828639,0.869126,03:19
2,0.198467,0.235039,0.923369,0.977826,0.856166,0.938939,0.810115,0.853011,03:19
3,0.195391,0.254530,0.922736,0.977138,0.855497,0.943122,0.790886,0.834817,03:19
4,0.190337,0.217293,0.928436,0.985812,0.865787,0.943337,0.825508,0.866988,03:19
5,0.173000,0.204108,0.929702,0.983333,0.868192,0.949167,0.820717,0.864125,03:19
6,0.166898,0.185442,0.936669,0.985573,0.882195,0.942886,0.886032,0.911136,03:19
7,0.156737,0.272644,0.908170,0.982495,0.836943,0.834023,0.905688,0.854406,03:19
8,0.163331,0.159023,0.944269,0.990432,0.896137,0.946243,0.906069,0.924445,03:20
9,0.162972,0.177326,0.939202,0.987658,0.886760,0.947462,0.873863,0.904983,03:19


## Batch Size

## 8

In [35]:
arch = resnet50
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [36]:
dl = db.dataloaders(df, bs=bs) 

In [37]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [38]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.434263,0.368028,0.875871,0.950123,0.768670,0.826443,0.809970,0.817713,03:01


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.294666,0.232131,0.920203,0.978535,0.850883,0.928050,0.840755,0.874575,04:13
1,0.291752,0.261036,0.922736,0.978626,0.855815,0.930412,0.828779,0.867908,04:12
2,0.274862,0.259119,0.914503,0.976920,0.844240,0.910842,0.843828,0.869148,04:12
3,0.273031,0.261926,0.916403,0.979261,0.843918,0.938330,0.798393,0.841458,04:12
4,0.286375,0.219842,0.921469,0.981994,0.853219,0.910946,0.859806,0.882328,04:13
5,0.249371,0.298663,0.923369,0.967612,0.856644,0.936805,0.782762,0.825440,04:12
6,0.294208,0.333818,0.913870,0.969424,0.838330,0.937334,0.746254,0.787218,04:13
7,0.213085,0.221667,0.929702,0.986443,0.868219,0.945167,0.843264,0.881935,04:12
8,0.279849,0.168746,0.936035,0.989454,0.882071,0.944100,0.900061,0.919191,04:13
9,0.281296,0.187444,0.934136,0.987760,0.877332,0.934607,0.888221,0.909172,04:12


## 16

In [47]:
arch = resnet50
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [48]:
dl = db.dataloaders(df, bs=bs) 

In [49]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [50]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.400648,0.459636,0.859405,0.948556,0.733876,0.899395,0.691698,0.729994,02:37


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.266218,0.241245,0.916403,0.979436,0.843474,0.919513,0.820386,0.856712,03:36
1,0.237252,0.213611,0.927802,0.985449,0.864658,0.937889,0.833264,0.871930,03:35
2,0.227198,0.241888,0.913870,0.983269,0.839744,0.924939,0.800397,0.840083,03:35
3,0.213895,0.209362,0.933502,0.978002,0.876642,0.928493,0.883033,0.902878,03:35
4,0.229692,0.269608,0.921469,0.982119,0.852851,0.941791,0.805848,0.851891,03:35
5,0.200232,0.334258,0.886637,0.974328,0.794584,0.918808,0.736152,0.772434,03:35
6,0.233891,0.228985,0.922736,0.981795,0.858136,0.917172,0.880141,0.896015,03:35
7,0.189099,0.251728,0.911970,0.975850,0.838448,0.847493,0.868580,0.855440,03:35
8,0.202870,0.156117,0.944269,0.990853,0.896759,0.947852,0.911240,0.927742,03:35
9,0.220237,0.159688,0.950602,0.989498,0.908171,0.952078,0.930533,0.940849,03:35


## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [59]:
arch = resnet50
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.06, reduction='mean', flatten=True, floatify=False, is_2d=True)
bs = 32
epoch = 30 

In [60]:
dl = db.dataloaders(df, bs=bs) 

In [61]:
learn = cnn_learner(dl, arch=arch, loss_func=loss_func, metrics=metrics)

In [62]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.571257,0.490882,0.868271,0.947639,0.751583,0.886067,0.736713,0.782956,02:26


epoch,train_loss,valid_loss,accuracy,roc_auc_score,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.419018,0.398985,0.913236,0.967394,0.837881,0.929449,0.776182,0.818342,03:20
1,0.382958,0.362010,0.927802,0.975623,0.864767,0.940667,0.834006,0.874245,03:19
2,0.354822,0.372639,0.920203,0.977628,0.850205,0.942107,0.788088,0.833033,03:19
3,0.356034,0.347481,0.934136,0.980949,0.876656,0.947670,0.844235,0.882900,03:20
4,0.348158,0.380431,0.920836,0.977985,0.854699,0.940952,0.818950,0.858161,03:20
5,0.334368,0.361153,0.925902,0.969684,0.861670,0.945094,0.796280,0.839856,03:19
6,0.330909,0.364478,0.923369,0.980276,0.859345,0.934495,0.869613,0.895544,03:20
7,0.319680,0.335258,0.938569,0.983865,0.885169,0.943794,0.875331,0.904261,03:20
8,0.317165,0.320195,0.942369,0.987524,0.892717,0.953177,0.888860,0.916016,03:20
9,0.318054,0.354550,0.933502,0.986577,0.875829,0.945844,0.833508,0.873090,03:20


# Efficientnet-B0

# Epoch

### 10

In [11]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
epoch = 10
bs = 32
loss_func=CrossEntropyLossFlat() 

In [13]:
dl = db.dataloaders(df, bs=bs) 

In [14]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [15]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.332779,0.613781,0.860038,0.733682,0.901387,0.642549,0.656213,02:36


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.210633,0.257930,0.923369,0.856138,0.940324,0.781659,0.828049,02:35
1,0.194218,0.327880,0.878404,0.787091,0.908792,0.774309,0.809162,02:35
2,0.201145,0.368455,0.908803,0.828753,0.930569,0.743249,0.789084,02:35
3,0.199135,0.193903,0.936035,0.882696,0.886954,0.925200,0.903576,02:35
4,0.173392,0.240729,0.925269,0.861573,0.948390,0.846734,0.886910,02:35
5,0.123314,0.173833,0.941102,0.890156,0.947612,0.864638,0.897444,02:35
6,0.107108,0.161002,0.947435,0.902247,0.957852,0.912759,0.933316,02:35
7,0.080860,0.127835,0.953135,0.912877,0.961864,0.919839,0.938816,02:35
8,0.084572,0.125697,0.958201,0.922277,0.964481,0.930062,0.946121,02:35
9,0.068999,0.121665,0.956935,0.919862,0.962791,0.929862,0.945246,02:35


## 20

In [24]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
epoch = 20
bs = 32
loss_func=CrossEntropyLossFlat() 

In [25]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [26]:
dl = db.dataloaders(df, bs=bs) 

In [27]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.311910,0.479080,0.896770,0.806062,0.925344,0.708552,0.741315,02:35


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.191999,0.180207,0.939835,0.887540,0.940360,0.870725,0.899854,02:35
1,0.173513,0.193175,0.929069,0.871121,0.854775,0.910526,0.876925,02:35
2,0.187337,0.244282,0.932236,0.872964,0.952133,0.841813,0.882842,02:35
3,0.180431,0.267163,0.920836,0.851870,0.947444,0.798820,0.847156,02:35
4,0.167334,0.240889,0.920203,0.850270,0.931323,0.811004,0.852092,02:35
5,0.187370,0.458107,0.893604,0.806342,0.923224,0.737522,0.773734,02:35
6,0.152596,0.196008,0.926536,0.863502,0.945328,0.835463,0.874240,02:35
7,0.160014,0.232747,0.937302,0.883674,0.955838,0.851482,0.892141,02:35
8,0.159003,0.289521,0.929702,0.868754,0.947865,0.795952,0.839705,02:35
9,0.112898,0.210326,0.937935,0.884109,0.953744,0.850379,0.890260,02:36


## 40

In [9]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
epoch = 40
bs = 32
loss_func=CrossEntropyLossFlat() 

In [10]:
dl = db.dataloaders(df, bs=bs) 

In [11]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [12]:
learn.fine_tune(epoch) 

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.332779,0.613781,0.860038,0.733682,0.901387,0.642549,0.656213,02:55


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.196330,0.196348,0.931602,0.872021,0.944574,0.825726,0.866704,02:35
1,0.165568,0.167081,0.940469,0.889041,0.939670,0.867587,0.896893,02:35
2,0.157777,0.272185,0.921469,0.852732,0.934993,0.766295,0.808547,02:35
3,0.164306,0.183432,0.938569,0.885083,0.943679,0.851309,0.887589,02:35
4,0.167469,0.238073,0.930336,0.869641,0.952505,0.812228,0.859024,02:35
5,0.135984,0.142413,0.946802,0.900856,0.961061,0.900551,0.926660,02:34
6,0.147727,0.244288,0.924636,0.861320,0.943577,0.838581,0.875384,02:34
7,0.140394,0.172925,0.941102,0.890509,0.929499,0.896423,0.911571,02:35
8,0.161632,0.248750,0.918303,0.853436,0.866932,0.890991,0.877586,02:35
9,0.171129,0.240449,0.929702,0.869662,0.943456,0.865445,0.898185,02:34


# Batch Size

## 8

In [21]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
bs = 8
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [22]:
dl = db.dataloaders(df, bs=bs) 

In [23]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [24]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.383026,0.844950,0.829639,0.682378,0.902666,0.598255,0.615977,03:43


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.217310,0.221159,0.925902,0.860978,0.938266,0.821787,0.863668,03:43
1,0.192999,0.230912,0.922736,0.855066,0.943868,0.795877,0.840572,03:42
2,0.179960,0.199253,0.929069,0.868692,0.929626,0.854709,0.884039,03:42
3,0.192055,0.218672,0.922736,0.855276,0.896864,0.799203,0.832952,03:42
4,0.211054,0.227703,0.925902,0.863971,0.910431,0.865022,0.883848,03:43
5,0.185028,0.297868,0.918936,0.847562,0.943814,0.775882,0.823207,03:42
6,0.225329,0.186205,0.934136,0.876634,0.943600,0.849226,0.885783,03:41
7,0.171188,0.333395,0.918936,0.848349,0.946874,0.777178,0.826444,03:41
8,0.237684,0.424683,0.893604,0.804348,0.923281,0.686705,0.704748,03:42
9,0.206796,0.183299,0.944269,0.897439,0.945293,0.902010,0.921457,03:44


## 16

In [33]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
bs = 16
epoch = 30 
loss_func=CrossEntropyLossFlat() 

In [34]:
dl = db.dataloaders(df, bs=bs) 

In [35]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [36]:
learn.fine_tune(epoch)

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.355093,1.278425,0.429386,0.175341,0.766809,0.373657,0.265584,02:57


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,0.196139,0.207408,0.930969,0.870852,0.922323,0.855290,0.883337,02:56
1,0.174329,0.191164,0.929702,0.868839,0.934911,0.850838,0.883675,02:56
2,0.173599,0.270657,0.922103,0.853580,0.945276,0.799015,0.845415,02:56
3,0.191466,0.215844,0.927169,0.863376,0.933471,0.813855,0.855751,02:56
4,0.186119,0.206846,0.930969,0.871695,0.938813,0.863611,0.895255,02:56
5,0.179190,0.204654,0.928436,0.865715,0.950151,0.809249,0.855383,02:56
6,0.191836,0.502558,0.890437,0.799729,0.921330,0.705888,0.733522,02:56
7,0.155538,0.240907,0.929702,0.869518,0.905230,0.865445,0.883343,02:56
8,0.195505,0.231356,0.920203,0.855849,0.913957,0.869206,0.886806,02:56
9,0.180110,0.228976,0.918303,0.846339,0.942713,0.786778,0.833994,02:56


## Loss Function 

### LabelSmoothingCrossEntropyFlat()

In [19]:
from efficientnet_pytorch import EfficientNet 
arch = EfficientNet.from_pretrained("efficientnet-b0")
bs = 32
epoch = 30
loss_func=LabelSmoothingCrossEntropyFlat(axis=-1, eps=0.2, reduction='mean', flatten=True, floatify=False, is_2d=True)

In [20]:
dl = db.dataloaders(df, bs=bs) 

In [21]:
learn = Learner(dl, model=arch, loss_func=loss_func, metrics=metrics)

In [22]:
learn.fine_tune(30) 

epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,2.148689,2.494614,0.835339,0.689222,0.897276,0.586119,0.584458,02:38


epoch,train_loss,valid_loss,accuracy,matthews_corrcoef,precision_score,recall_score,f1_score,time
0,2.051933,2.031061,0.933502,0.876100,0.905072,0.874721,0.888783,02:37
1,2.027956,2.028454,0.936035,0.880311,0.946889,0.855531,0.891809,02:37
2,2.005783,2.034451,0.935402,0.879501,0.947497,0.843877,0.881718,02:36
3,2.017411,2.099018,0.917036,0.845750,0.929758,0.750720,0.788753,02:36
4,2.011442,2.042567,0.934136,0.877303,0.947928,0.858100,0.894459,02:36
5,2.020347,2.022962,0.943635,0.895084,0.950141,0.886841,0.913553,02:37
6,2.016466,2.070476,0.930969,0.872774,0.953904,0.845317,0.887423,02:36
7,2.002652,2.022348,0.941735,0.892147,0.920047,0.921934,0.920979,02:36
8,2.015983,2.011866,0.940469,0.891158,0.937367,0.924330,0.929458,02:36
9,1.999658,2.084357,0.924636,0.859465,0.927233,0.809757,0.848580,02:37
